In [18]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import SimpleITK as sitk
import skimage.io as io

In [8]:
def read_image_into_ndArray(imagefile, PlugIn):
    imageArray = io.imread(imagefile, plugin=PlugIn)
    print("The dimension of the image is ", imageArray.shape)
    return imageArray

## 2D images (jpg, png, etc)

In [9]:
imgfile = "/home/seonwhee/Deep_Learning/Solaris/week9/FCN-BRATS/Data_zoo/BRATS/images/training/VSD.Brain.XX.O.MR_T1c.54514_1.jpg"
img2 = read_image_into_ndArray(imgfile, PlugIn='matplotlib')

The dimension of the image is  (240, 240, 3)


## ITK metaImage

In [10]:
mhafile = "/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0001_1/VSD.Brain.XX.O.MR_Flair.54512/VSD.Brain.XX.O.MR_Flair.54512.mha"
img = read_image_into_ndArray(mhafile, PlugIn='simpleitk')

The dimension of the image is  (155, 240, 240)


In [24]:
mhafile2 = "/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat105_0001/VSD.Brain.XX.O.MR_T1.35536/VSD.Brain.XX.O.MR_T1.35536.mha"
img2 = read_image_into_ndArray(mhafile2, PlugIn='simpleitk')

The dimension of the image is  (155, 240, 240)


#### Multiple files

In [ ]:
mhafiles = glob('/home/seonwhee/Deep_Learning/Datasets/BRATS-2/Image_Data/HG/*/*/*mha')

In [ ]:
imgList = []
for a_file in mhafiles:
    filename = a_file.split("/")[-1]
    imgInput = sitk.ReadImage(a_file)
    nda = sitk.GetArrayViewFromImage(imgInput) # Get a view of the image data as a numpy array, useful for display
    img = sitk.GetImageFromArray(nda)
    imgList.append((filename, img))
    

In [ ]:
def figure_images(imageList, z):
    
    i = 1
    for a_file in imageList:
        sliced = sitk.GetArrayViewFromImage(a_file[1])[z,:,:]        
        #plt.subplot(round(len(imageList)/3)+1,3, i)
        plt.figure()
        plt.title(a_file[0])
        plt.imshow(sliced, cmap=plt.cm.Greys_r)
        plt.axis('off')
        i = i+1


In [ ]:
figure_images(imgList, 80)

### T1 and T2-weighted image

In [21]:
from ipywidgets import interact, FloatSlider
from myshow import myshow, myshow3d

In [23]:
img_T1 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1.54567/VSD.Brain.XX.O.MR_T1.54567.mha")
img_T2 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T2.54569/VSD.Brain.XX.O.MR_T2.54569.mha")
# To visualize the labels image in RGB with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)
img_T2_255 = sitk.Cast(sitk.RescaleIntensity(img_T2), sitk.sitkUInt8)

myshow3d(img_T1)
myshow3d(img_T2)

A Jupyter Widget

A Jupyter Widget

## dicom files

In [5]:
import cv2
import numpy as np
import dicom as pdicom
import os
import subprocess as sp

In [3]:
dicomfiles = glob("/home/seonwhee/Deep_Learning/Datasets/*/*dcm")

In [ ]:
def View_dicom(dicomList):
    for a_dicom in dicomList:
        an_array = pdicom.read_file(a_dicom)
        ConstPixelDims = (int(an_array.Rows), int(an_array.Columns), len(dicomfiles))
        ConstPixelSpacing = (float(an_array.PixelSpacing[0]), float(an_array.PixelSpacing[1]), float(an_array.SliceThickness))
        x = np.arange(0.0, (ConstPixelDims[0]+1)*ConstPixelSpacing[0], ConstPixelSpacing[0])
        y = np.arange(0.0, (ConstPixelDims[1]+1)*ConstPixelSpacing[1], ConstPixelSpacing[1])
        z = np.arange(0.0, (ConstPixelDims[2]+1)*ConstPixelSpacing[2], ConstPixelSpacing[2])
        ArrayDicom = np.zeros(ConstPixelDims, dtype=an_array.pixel_array.dtype)        
        ArrayDicom[:, :, dicomfiles.index(a_file)] = an_array.pixel_array
        plt.figure()
        plt.axes().set_aspect('equal', 'datalim')
        plt.set_cmap(plt.gray())
        plt.pcolormesh(x,y,np.flipud(ArrayDicom[:,:,80]))
        

In [ ]:
def applying_N3(inputList):
    output_dir = "/home/seonwhee/Radiology/N3_output/"
    command = "/home/seonwhee/Radiology/itkN3 %s %s" %(inputList, output_dir)
    sp.call(cmd)
    